# AIRQUALITY DATASET EXPLORATION

In [ ]:
!pip install miceforest
{'name':'Álvaro Riobóo de Larriva',
'start_date':'2021/03/09'}

In [ ]:
#--< main modules >--#
import numpy as np 
import pandas as pd 
import scipy as sp
import statsmodels.api as sm
import sklearn

#--< visualization >--#
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotnine as p9

#--< others >--#
from datetime import datetime
import time
from IPython.display import display 
import gc

for i in locals().copy():
    try:
        print("%s:"%eval(i).__name__, eval(i).__version__)
    except:
        continue

### REFERENCE LINKS:
- [UCI ML Airquality Dataset](https://archive.ics.uci.edu/ml/datasets/Air+Quality) : Descripción de los datos
- [AirQualityUCI.csv](https://github.com/shrikumarp/airquality/blob/master/AirQualityUCI.csv) : Repositorio con los datos en CSV. Usaremos la versión raw para leer desde GitHub.


*Columns:*
Index(['Date', 'Time', 'CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)',
       'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)',
       'PT08.S5(O3)', 'T', 'RH', 'AH'],
      dtype='object')
      
**Data Set Information:**

The dataset contains 9358 instances of hourly averaged responses from an array of 5 metal oxide chemical sensors embedded in an Air Quality Chemical Multisensor Device. The device was located on the field in a significantly polluted area, at road level,within an Italian city. Data were recorded from March 2004 to February 2005 (one year)representing the longest freely available recordings of on field deployed air quality chemical sensor devices responses. Ground Truth hourly averaged concentrations for CO, Non Metanic Hydrocarbons, Benzene, Total Nitrogen Oxides (NOx) and Nitrogen Dioxide (NO2) and were provided by a co-located reference certified analyzer. Evidences of cross-sensitivities as well as both concept and sensor drifts are present as described in De Vito et al., Sens. And Act. B, Vol. 129,2,2008 (citation required) eventually affecting sensors concentration estimation capabilities. Missing values are tagged with -200 value.
This dataset can be used exclusively for research purposes. Commercial purposes are fully excluded.

**Attribute Information:**

0. Date (DD/MM/YYYY)
1. Time (HH.MM.SS)

2. True hourly averaged concentration CO in mg/m^3 (reference analyzer)
3. PT08.S1 (tin oxide) hourly averaged sensor response (nominally CO targeted)

4. True hourly averaged overall Non Metanic HydroCarbons concentration in microg/m^3 (reference analyzer)
> 5. True hourly averaged Benzene concentration in microg/m^3 (reference analyzer)
6. PT08.S2 (titania) hourly averaged sensor response (nominally NMHC targeted)

7. True hourly averaged NOx concentration in ppb (reference analyzer)
8. PT08.S3 (tungsten oxide) hourly averaged sensor response (nominally NOx targeted)

9. True hourly averaged NO2 concentration in microg/m^3 (reference analyzer)
10. PT08.S4 (tungsten oxide) hourly averaged sensor response (nominally NO2 targeted)

> 11. PT08.S5 (indium oxide) hourly averaged sensor response (nominally O3 targeted)

12. Temperature in Â°C
13. Relative Humidity (%)
14. AH Absolute Humidity 

In [ ]:
###----------------< START of 'airquality_nb.ipynb'>---------------###

## 0. LOAD DATASET AND PARAMETERS

In [ ]:
# DATASET LOADING
airq = pd.read_csv("https://raw.githubusercontent.com/shrikumarp/airquality/master/AirQualityUCI.csv",
                  delimiter=";", decimal=",", 
                  usecols=range(15)) # dropna deletes empty lines, we need NAN the -200 values.
airq = airq.dropna().replace(-200,np.nan) # 1 more fix after

print('Need to fix columns names a bit: %a\n'%airq.columns)
airq.columns = [c[0] for c in airq.columns.str.split("(")]




In [ ]:
# DATA CLEANING & PARAMETERS
##--<INDEX>--## # TIME_INDEX, drop rest of time-referers
try:
    time_index = pd.Series([datetime.strptime(i, "%d/%m/%Y%H.%M.%S") for i in airq['Date'] + airq['Time']])
    airq.insert(0, 'time_index', time_index)
    airq = airq.drop(['Time','Date'], axis=1)
except Exception as e:
    print('Ya se ha realizado la operacion anteriormente\nError:',e)

date_range = (airq['time_index'].min(), airq['time_index'].max())
print(date_range)

##---<DTYPE COLS>---##
numeric_cols = airq.select_dtypes(np.float64).columns # all numericals were parsed as np.float64
date_cols = ['time_index']                           # time_index from dataset
sensor_cols = airq.columns[airq.columns.str.startswith('PT08')]
abundance_cols = ['CO','NMHC','NOx','NO2','C6H6']    # last 'C6H6' is not measured, they are put in order with sensor_cols
airprops_cols = ['T','RH','AH']                      # air properties
sensor_refs = [par for par in zip(sensor_cols,abundance_cols[:-1])]

##---<PRINTS>---##
for i in airq.isna().sum():
        print("NAN: %5i    p: %.2f "%(i,i/len(airq)))
        
print('\nSum NAN values:%i'%airq.isna().sum().sum())
print('\nNew column names:%a\n'%airq.columns)
display(airq.head())

In [ ]:
sensor_refs

## 1. EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
# IMPUTE MISSING DATA WITH MICEFOREST -> MICE AND RANDOM FOREST GUESSING.
import miceforest as mf

airq_amp = mf.ampute_data(airq.reset_index()[numeric_cols], random_state=1994)

# Create kernel
kds = mf.KernelDataSet(
  airq_amp,
  save_all_iterations=True,
  random_state=1994
)

# Run the MICE algorithm for 3 iterations
kds.mice(3)

# Return the completed kernel data
imputed_data = kds.complete_data()

gc.collect()

print('Before imputation:%a'%airq.isna().sum())
print('Total NAN values after imputation:', imputed_data.isna().sum().sum())

airq[numeric_cols] = imputed_data

airq.set_index('time_index', inplace=True)


In [ ]:
# Display plain information, types and main stats.
descr = airq.describe()
nunique_values = airq.apply( lambda x: x.nunique())

airq.info()
print('number of unique values:\n%a'%nunique_values)
display(descr)

qlabels = ['min','25%', '50%', '75%', 'max']
qvals = descr.loc[qlabels] # we keep quartile values from df description 
statslabels = ['count','mean','std']
statsvals = descr.loc[statslabels] # also keep mean statistics
 

In [ ]:
# CORRPLOT
mask = np.zeros_like(airq.corr(), dtype=np.bool)
mask[np.tril_indices_from(mask)] = True

plt.figure(figsize=(20,5))
sns.heatmap(airq.corr(), mask=np.triu(np.ones_like(airq.corr())), cmap=plt.cm.viridis, annot=True, fmt=".2f")

Podemos ver información interesante en nuestra matriz de correlación. Comentaremos la respuesta del sensor frente a la concentración que pretende medir.

- PT08.S1(C0)/C0:     0.88 (BUENA)
- PT08.S2(NMHC)/NMHC: 0.90 (BUENA)
- PT08.S3(NOx)/NOx:  -0.66 (ANTICORR)
- PT08.S4(NO2)/NO2:   0.19  (POBRE)
- PT08.S5(O3): Buena correlación en torno a las demás concentraciones y sensores, no hay referencia de O3. No correlaciona con AH y poco con RH y T.
- C6H6: Buena correlación en torno a las demás concentraciones y sensores, no hay sensor de C6H6. No correlaciona con las propiedades del aire (T,RH,AH).
- Las propiedades del aire no son buenos predictores de las demás variables, ni siquiera se correlacionan entre sí.

In [ ]:
# PAIRPLOT
sns.pairplot(airq[sensor_cols])
sns.pairplot(airq[abundance_cols])
sns.pairplot(airq[airprops_cols])

Por un estudio similar [Fig 3., pag 1524](https://www.researchgate.net/publication/311459930_Summertime_ambient_ammonia_and_its_effects_on_ammonium_aerosol_in_urban_Beijing_China) realizado en China, vemos que han aproximado las curvas de concentración de gases a dos Lorentzianas, una para el aire limpio y otra para el contaminado, con buenos resultados. De hecho, se pueden ver dos picos propios de las dos distribuciones en la mayoría de gráficas de este tipo. Un ajuste de estas curvas se expera que dé como resultado los picos (~ medias de la distribución) en un aire limpio y otro contaminado.

- En los gráficos de sensores, se aprecia una correlación positiva entre todos ellos, menos en el caso del S3, que anticorrelaciona con los demás.

- En los gráficos de concentraciones de gases, se aprecia una correlación positiva y clara entre todos ellos.

- En los gráficos de propiedades del aire, se aprecia una correlación positiva, aunque también una dependencia entre la humedad relativa y la absoluta.

In [ ]:
# NOx vs. S5(NOx)
sns.lmplot(x='PT08.S3',y='NOx', scatter_kws={'color':'grey','alpha':.6}, data=airq,line_kws={'color':'green'})  # Vemos que efectivamente NOx y su sensor tienen correlación negativa.
plt.title("NOx & sensor S5(NOx)")

In [ ]:
# AH vs. RH
normalize = lambda x: (x-x.mean())/x.std()

fig,axs = plt.subplots(1,3, figsize=(10,5))
with plt.style.context('seaborn'):
    airq.plot(kind='scatter', x='RH', y='AH', ax=axs[0],
             color="red" )
    axs[0].set_title('Scatter plot AH/RH')
    
    airq.plot(kind="hist", y='AH', ax=axs[1], 
              bins=50, fill='black', density=True, grid=True, rwidth=0.5)
    axs[1].set_xlabel('AH')
    axs[1].set_title('Histogram of AH')
    
    airq.plot(kind="hist", y='RH', ax=axs[2], 
              bins=50, fill='black', density=True, grid=True, rwidth=0.5, color="green")
    axs[2].set_xlabel('RH')
    axs[2].set_title('Histogram of RH')

In [ ]:
# HISTOGRAMS
fig, axs = plt.subplots(1,2, figsize=(10,5))
airq.plot(kind="hist", bins=30, density=True, alpha=0.7, ax=axs[0])
axs[0].set_title('Histogram of numeric variables')
airq.plot(kind="hist", bins=30, density=True, alpha=0.7, stacked=True, ax=axs[1])
axs[1].set_title('Histogram of numeric variables (stacked)')

In [ ]:
# DENSITYPLOT of normalized dataset.
fig ,ax = plt.subplots(figsize=(20,10))
normalize(airq).plot(kind="density", ax=ax,
                    style='-*', ms=0.6, cmap=plt.cm.Accent)
plt.axvline(x=0,**{'marker':'^','color':'black'})

En el gráfico anterior normalizado, solemos ver mayormente poblaciones sesgadas a la izquierda de su media (centrada en 0). Vemos algunas distribuciones bimodales, pero sobre todo se pueden  observar exponenciales negativas, en teoría correspondientes a la referencia de las concentraciones de los gases que estamos midiendo. La ley que describe el comportamiento a grandes rasgos de la concentración de un gas es: $\frac{dN}{dt} = -\lambda N \rightarrow N(t)=N_0 e^{-t/\tau}$, que nos dice que la concentración disminuye proporcionalmente a la concentración en ese momento, por lo que esperamos que estas distribuciones sean exponenciales negativas en el tiempo ($\tau$ es el tiempo de vida medio, por el cual la concentración disminuye en $1/e$ de su valor original). Podríamos más adelante ajustar a las generalizables funciones gamma.

In [ ]:
# DENSITYPLOT dists of gases:
fig, axs = plt.subplots(1,2, figsize=(20,5), sharey=True)
normalize(airq[abundance_cols]).plot(kind="density", ax=axs[0])
axs[0].axvline(x=0, color='black', marker="|" ,linestyle="--")
axs[0].set_title(r'Distributions: Concentration of gas')

normalize(airq[sensor_cols]).plot(kind="density", ax=axs[1])
axs[1].axvline(x=0, color='black', marker="|" ,linestyle="--")
axs[1].set_title(r'Distributions: Sensor measurements of gas')



Podemos ver en los gráficos anteriores que la respuesta de los sensores a las abundancias las tendencias bimodales son inapreciables y en general se parece más a una única distribución normal. La referencia posee picos mucho más puntiagudos, mientras que los sensores parecen diseñados para tener una respuesta normal.

Veremos ahora las propiedades del aire (Temperature, Relative Humidty, Absolute Humidity). Investigamos ésto un poco más en ventanas temporales.

In [ ]:
# AIR PROPERTIES
rolling_freqs = ['24h', '7d','30d','120d','365d']
airprops = airq[airprops_cols]

with plt.style.context('seaborn-notebook'): 
    
    fig, axs = plt.subplots(1,len(rolling_freqs), figsize=(15,4))   
    for i,freq in enumerate(rolling_freqs):
        airprops.rolling(freq).mean().plot(ax=axs[i], grid=True)
        axs[i].set_title("Air_props\ win:%s"%freq)
        labels = axs[i].get_xticks()
        plt.setp(axs[i].xaxis.get_majorticklabels(), rotation=90 )
        
    fig, axs = plt.subplots(1,len(rolling_freqs), figsize=(15,4))
    for i,freq in enumerate(rolling_freqs): 
        normalize(airprops).rolling(freq).mean().plot(ax=axs[i], grid=True)
        axs[i].set_title("N(0,1) Air_props\ win:%s"%freq)
        plt.setp(axs[i].xaxis.get_majorticklabels(), rotation=90 )
        
gc.collect()

> Hemos escogido ventanas temporales para 1 dia,1 semana (7d), 1 mes (30d), 4 meses (120d), 1 año (365d). Se puede observar que nuestros datos son insuficientes para mostrar un ciclo estacional completo. Se estima que hay un ciclo de alrededor de 1 año, pero como tenemos una muestra temporal escasa y afectada gravemente por la tendencia positiva, no podemos concluirlo a simple vista. 

Habría que realizar un modelo de autoregresión y media móvil **(ARIMA)** para poder estimar la tendencia y estacionalidad de una manera más rigurosa. Se pueden valorar a simple vista 3 frecuencias naturales de nuestros datos:

- ~ 1-2 semanas
- ~ 1 mes
- ~ 1 año
    

In [ ]:
hour_fraction_crit_values = []
for crit_value in [50,100,150,200]:
    hour_fraction_crit_values.append(airq['NO2'].where(airq['NO2'] >=crit_value).count()/len(airq))
print(hour_fraction_crit_values)

Éste dataset no tiene un target pre-establecido. Sin embargo, el S4 no correlaciona con la concentración **NO2** para el cual está diseñado. Siendo el dióxido de nitrógeno un gas peligroso para el ser humano y proveniente de vehículos, plantas eléctricas, emisión industrial, construcción, y en definitiva de combustibles fósiles, debemos intentar medir y predecir la concentración del gas y pretender que se sitúe en un rango razonable. 
    - (Bueno: 0-50, Moderado: 50-100, Peligroso para grupos sensibles: 100-150, Insano: 150-200). 
    - Mean: 111.4 , Max: 340
Según el output de la celda anterior, aprox. un ~21% del tiempo el nivel se sitúa por encima de la calificación de Peligroso, y un ~4% de éste en el nivel Insano. Ésto podría afectar gravemente a la población que viva o transite por éste lugar a lo largo del día. Por ello, **NO2 será nuestra variable objetivo en este estudio**, aunque también podría estudiarse la concentración de **NOx**, otro gas tóxico y peligroso.

## 2. FINDING CANDIDATE ALGORITHMS

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import RandomizedSearchCV, train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error  


In [ ]:
# DATA/TARGET definition
data = airq.reset_index().drop(['time_index','NO2'], axis=1)
target = airq.reset_index().drop('time_index', axis=1)['NO2']

# TRAIN/TEST subsets
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=1994)

# SCALER
rob = RobustScaler()
X_train = rob.fit_transform(X_train)
X_test = rob.fit_transform(X_test)

Debemos estandarizar nuestro dataset, es decir, convertir nuestros datos a rangos y dispersión similares. Usaremos el RobustScaler(), puesto que nos ayudará a minimizar el impacto de los outliers sustrayendo la mediana y dividiendo por el rango intercuartil de nuestros datos.

La variable target será **NO2** como se tratará de justificar un poco más adelante.

Para la búsqueda de algoritmos candidatos, usaremos los siguientes para el problema de regresión:

- LinearRegression()
- KNeighborsRegressor()
- DecisionTreeRegressor()
- RandomForestRegressor()
- SVR()
- XGBRegressor()




In [ ]:
# We define a function to evaluate all models in a dictionary.
def train_all_models(models_dict, train_now=True):
    if train_now:
        for k,m in models_dict.items():
            start_time = time.time()
            print('MODEL:%s'%k)
            m.fit(X_train, y_train)
            print("(model_runtime= %.2f s)\n"%(time.time() - start_time))
    else: 
        print("Not training for now")
    gc.collect()
    return models_dict

def plot_predict_all_models(models_dict):
    for k,m in models_dict.items():
        y_pred = m.predict(X_test)
        print('MODEL :%s\n'%k, 
              'R^2= %.3f\n'%r2_score(y_test, y_pred),
              'MSE= %.3f\n'%mean_squared_error(y_test, y_pred) , 
              'MAE= %.3f\n'%mean_absolute_error(y_test, y_pred))
        fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
        axs[0].set_title("Predicted/Observed")
        axs[0].set_xlabel("Observed") ; axs[0].set_ylabel("Predicted")
        axs[0].scatter(x=y_test, y=y_pred, color="g", marker="o", alpha=0.3)

        axs[1].set_title("Error distribution")
        axs[1].set_xlabel("X"); axs[1].set_ylabel("Y")
        axs[1].hist( y_pred-y_test, bins=30, rwidth=0.6, density=True)
    gc.collect()

In [ ]:
%timeit
# MODELS WITH NO PARAMETERS (VANILLA)

Lr = LinearRegression()
KNr = KNeighborsRegressor()
DTr = DecisionTreeRegressor()
RFr = RandomForestRegressor()
SVr = SVR()
XGBr = XGBRegressor()

models_vanilla = {'Lr':Lr,
                 'KNr':KNr,
                 'DTr':DTr,
                 'RFr':RFr,
                 'SVr':SVr,
                 'XGBr':XGBr}



models_vanilla = train_all_models(models_vanilla)

In [ ]:
## EVALUATION VANILLA_MODELS
plot_predict_all_models(models_vanilla)

Vemos que nuestros modelos 'vanilla' mejor parados han sido:
- Random Forest: $R^2=0.886$
- XGBoost: $R^2=0.869$
- K-Nearest Neighbours: $R^2=0.859$

## 3. EVALUATION AND IMPROVEMENTS FOR THOSE ALGORITHMS

Para la optimización de nuestros hiperparámetros, usaremos "RandomizedSearchCV" de sklearn para quedarnos con la mejor combinación posible de hiperparámetros para cada algoritmo.

Expondremos gráficos y los resultados principales de $R^2$ de nuestros modelos con los datos. 

Guardaremos los resultados de los modelos anteriores ('train_accuracy','test_accuracy') en un DataFrame y exportaremos a csv. Exportaremos también el mejor modelo con formato 'pickle'.

In [ ]:
%timeit
# MODELS AND KEYWORD PARAMETERS DEFINED: RANDOMIZEDSEARCHCV  # NOTE: THIS IS THE MOST TIME-DEMANDING
Lr_kw = {'fit_intercept':[True,False],
        'normalize':[True,False],
        'positive':[True,False]}
Lr_cv = RandomizedSearchCV(Lr, Lr_kw, 
                           cv=4, random_state=1994)

KNr_kw = {'n_neighbors':np.arange(1,13),
          'weights':['uniform','distance'],
          'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
          'leaf_size':np.arange(12,30)}
KNr_cv = RandomizedSearchCV(KNr, KNr_kw, 
                            cv=5, random_state=1994)

DTr_kw = {"splitter" : ["best","random"], 
        "min_samples_leaf" : np.arange(1,9), 
        "max_features" : ["auto","sqrt","log2", None], 
        "max_depth" : [3,4,5,None], 
        "criterion" : ['mse',"mae","poisson"]}
DTr_cv = RandomizedSearchCV(DTr, DTr_kw,
                           cv=5, random_state=1994)
          
RFr_kw = {"n_estimators" : [50,75,100], 
          "min_samples_leaf" : np.arange(1,7, 2), 
          "max_features" : ["auto","sqrt","log2"], 
          "max_depth" : [4,5,6], 
          "criterion" : ['mse','mae']}
RFr_cv = RandomizedSearchCV(RFr, RFr_kw,
                           cv=4, random_state=1994)
          
SVr_kw = {"kernel" : ["linear","poly","rbf","sigmoid"],
          "degree": [2, 3, 4],
          "gamma" : ["scale","auto"]
         }
SVr_cv = RandomizedSearchCV(SVr, SVr_kw,
                           cv=5, random_state=1994)
          
XGBr_kw = {'n_estimators' : [100, 500, 1000],
    'max_depth' : [2, 3, 5, 10],
    'learning_rate' : [0.05, 0.1, 0.15],
    'min_child_weight' : [1, 2, 3, 4],
    'booster' : ['gbtree','gblinear'],
    'base_score' : [0.25, 0.5, 0.75, 1]}
XGBr_cv = RandomizedSearchCV(XGBr, XGBr_kw,
                            cv=3, n_iter=5, 
                            scoring = 'neg_mean_absolute_error',
                            n_jobs = -1,
                            verbose = 5, 
                            return_train_score = True,
                            random_state=1994)


models_CV = {'Lr':Lr_cv,
            'KNr':KNr_cv,
            'DTr':DTr_cv,
            'RFr':RFr_cv,
            'SVr':SVr_cv,
            'XGBr':XGBr_cv}

models_CV = train_all_models(models_CV, train_now=True)
        

In [ ]:
# PRINT BEST_PARAMS FOUND BY RANDOM SEARCH
for k,v in models_CV.items():
    print("Model:%s"%k,"\nBest_parameters:%a"%v.best_params_)
    print('\n')

In [ ]:
%timeit
### MODELS AFTER HYPERPARAMETERS TUNING: IMPROVED
Lr_imp = LinearRegression(**models_CV['Lr'].best_params_) 
KNr_imp = KNeighborsRegressor(**models_CV['KNr'].best_params_)
DTr_imp = DecisionTreeRegressor(**models_CV['DTr'].best_params_)
RFr_imp = RandomForestRegressor(**models_CV['RFr'].best_params_)
SVr_imp = SVR(**models_CV['SVr'].best_params_)
XGBr_imp = XGBRegressor(**models_CV['XGBr'].best_params_)

models_improved = {"Lr" : Lr_imp,
                  "KNr" : KNr_imp,
                  "DTr" : DTr_imp,
                  "RFr" : RFr_imp,
                  "SVr" : SVr_imp,
                  "XGBr" : XGBr_imp}

models_improved = train_all_models(models_improved)

In [ ]:
## EVALUATION CV_MODELS
plot_predict_all_models(models_improved)

In [ ]:
%timeit
### BAGGING ENSEMBLE TO ALL METHODS
Lr_bag = BaggingRegressor(LinearRegression(**models_CV['Lr'].best_params_)) 
KNr_bag = BaggingRegressor(KNeighborsRegressor(**models_CV['KNr'].best_params_))
DTr_bag = BaggingRegressor(DecisionTreeRegressor(**models_CV['DTr'].best_params_))
RFr_bag = BaggingRegressor(RandomForestRegressor(**models_CV['RFr'].best_params_))
SVr_bag = BaggingRegressor(SVR(**models_CV['SVr'].best_params_))
XGBr_bag = BaggingRegressor(XGBRegressor(**models_CV['XGBr'].best_params_))


models_bag = {"Lr" : Lr_bag,
             "KNr" : KNr_bag,
             "DTr" : DTr_bag,
             "RFr" : RFr_bag,
             "SVr" : SVr_bag,
             "XGBr" : XGBr_bag}

# TRAIN
models_bag = train_all_models(models_bag, train_now=True)

In [ ]:
## EVALUATION BAG_MODELS
plot_predict_all_models(models_bag)



In [ ]:
%timeit
# MAIN RESULTS INTO DATAFRAMES
algorithms = ['Linear Regression',
            'K-Nearest Neighbour Regressor',
            'Decision Tree Regressor', 
            'Random Forest Regressor',
            'Support Vector Machine Regressor',
            'XGBoost']

vanilla_results = pd.DataFrame( {"Algorithm": algorithms,
                          "Train_score":[m.score(X_train,y_train) for m in models_vanilla.values()],
                          "Test_score":[m.score(X_test,y_test) for m in models_vanilla.values()]
                                }).sort_values('Test_score', ascending=False)

improved_results = pd.DataFrame({"Algorithm": algorithms,
                          "Train_score":[m.score(X_train,y_train) for m in models_improved.values()],
                          "Test_score":[m.score(X_test,y_test) for m in models_improved.values()]
                                }).sort_values('Test_score', ascending=False)

bag_results = pd.DataFrame({"Algorithm": algorithms,
                          "Train_score":[m.score(X_train,y_train) for m in models_bag.values()],
                          "Test_score":[m.score(X_test,y_test) for m in models_bag.values()]
                               }).sort_values('Test_score', ascending=False)

display("VANILLA:", vanilla_results)
display("IMPROVED:", improved_results)
display("BAGGING:", bag_results)

In [ ]:
# EXPORT CLEANED AND IMPUTED DATASET TO CSV (for production)
airq.to_csv("airq.csv", header=True)

# EXPORTS TO CSV
vanilla_results.to_csv("airq_vanilla_results.csv")
improved_results.to_csv("airq_improved_results.csv")
bag_results.to_csv("airq_bagging_results.csv")

# EXPORT TO PICKLE
import pickle
model_filename = 'airq_xgboost_bag.sav'
pickle.dump(models_bag['XGBr'], open(model_filename, 'wb'))



In [ ]:
# FEATURE IMPORTANCES
feature_importances = pd.Series(models_improved['XGBr'].feature_importances_).sort_values(ascending=False)
feature_importances.index = feature_importances.index.map({ i:col for i,col in enumerate(data.columns)})
display(feature_importances)

Podemos ver que el la referencia de 'NOx' resulta ser el mejor predictor, de lejos los sigue la humedad absoluta "AH", el gas "CO" y la humedad relativa ("RH"). El sensor de "NHMC" (PT08.S2) los sigue en importancia.

Como adelantábamos por el mapa de correlación, el sensor asociado a "NO2" tiene pobre relevancia como predictor del mismo.

## <ins>*CONCLUSIONES*:</ins>

- 1. ANÁLISIS EXPLORATORIO DE DATOS:

    - La correlación de Pearson entre variables indica lo siguiente: S1(CO) y S2(NMCH) son buenos predictores lineales de sus concentraciones, S3(NOx) está anticorrelacionado con su variable, sin embargo, la transformación -np.log10(S3) llega a correlacionar 0.75 con NOx, S4 no correlaciona con NO2.
    
    - Los histogramas indican que las concentraciones siguen de forma más o menos diferenciada una distribución con dos jorobas, pudiéndose ajustar por dos lorentzianas diferentes, con un pico de aire limpio y otro de aire poluto. Los sensores y su respuesta promediada cada hora dan poca cuenta de este comportamiento, y se ajustan mejor en el caso de las concentraciones que no tienen picos muy altos y que pueden ajustarse a una distribución normal.
    
    - Las propiedades del aire (Temperatura, Humedad relativa y Humedad absoluta) no son buenos predictores de las demás variables, ni siquiera se correlacionan demasiado entre sí. La temperatura y humedad absoluta se correlacionan positivamente, y éstas negativamente con la humedad relativa.
    
    - Las propiedades del aire son muy cambiantes y siguen ciclos establecidos. Probablemente los ciclos de estacionalidad y tendencia relacionados con la serie temporal sean los que expliquen la variabilidad en las propiedades del aire. Por ejemplo, si estamos en verano la humedad absoluta, relativa y la temperatura serán consistentemente diferentes a las del invierno. Hemos distinguido visualmente 3 frecuencias para los datos (1-2 dias, 1-2 semanas, 1año), lo cual podría investigarse con modelos ARIMA.
    
- 2. BUSCANDO ALGORITMOS CANDIDATOS:

Éste dataset no tiene un target pre-establecido. Sin embargo, el S4 no correlaciona con la concentración **NO2** para el cual está diseñado. Siendo el dióxido de nitrógeno un gas altamente peligroso para el ser humano y proveniente de vehículos, plantas eléctricas, emisión industrial, construcción, y en definitiva de combustibles fósiles, debemos intentar medir y predecir los valores de éste gas y que se sitúen en un rango razonable. 
    
    - (Bueno: 0-50, Moderado: 50-100, Peligroso para grupos sensibles: 100-150, Insano: 150-200). 
    - Nuestros datos indican lo siguiente:      Mean=111.4 , Max=340

Por tanto, hemos definido nuestro target como 'NO2', separando el resto de datos (de los cuales hemos quitado también el tiempo).

Hemos estandarizado nuestros datos con un escalador de mediana y rango intercuartil para minimizar el impacto de valores extremos. Luego, hemos separado nuestros datos en 'train/test' (sin validación cruzada para una mayor simplicidad), con una fracción de test del 30%.

Hemos usado los siguientes y famosos algoritmos de sklearn para problemas de regresión:

    - LinearRegression(): Modelo más simple, ajusta los coeficientes por medio de minimizar diferencias cuadráticas.
    - KNeighborsRegressor(): Modelo sencillo que, una vez se establece k (el número de predictores máximo), nos dará la importancia de predictores en torno a la variable de regresión.
    - DecisionTreeRegressor(): Modelo sencillo que penaliza la distancia de nuestros datos en relación a valores de prueba , construye un árbol de decisión y permite realizar una regresión. Permite ver la importancia de predictores.
    - RandomForestRegressor(): Modelo mejorado basado en árboles de decisión que permite promediar una muestra significativa de ellos, sus resultados, y ver la importancia de predictores.
    - SVR(): Modelo de máquinas de soporte vectorial ('Support Vector Machines'), que busca hacer la mejor división de nuestros datos mediante hiperplanos que los separen.
    - XGBRegressor(): Modelo complejo y óptimo ('eXtreme Gradient Boosting') basado en árboles de decisión, optimización del descenso del gradiente y una randomizazión de parámetros optimizada, entre otros.

Hemos usado 'RandomizedSearchCV' para buscar y quedarnos con los mejores hiperparámetros de nuestros algoritmos. 

**NOTA: Hemos evaluado el $R^2$ (score) del modelo y otras métricas, también hemos dibujado el gráfico de dispersión de la predicción respecto al objetivo, y el histograma del error asociado. Ésto lo hemos hecho para todos los modelos.**

- 3. EVALUACIÓN Y MEJORAS PARA ÉSTOS ALGORITMOS:

Una vez buscado los mejores parámetros para nuestros modelos, hemos evaluado nuestros modelos con éstos, lo que hemos llamado modelos "improved".

Luego, hemos usado un método de ensamblado('BaggingRegressor') con los originales para mejorar nuestros modelos con parámetros optimizados, lo que hemos llamado modelos "bagging".

Los algoritmos principales evaluados en test, tanto para los modelos "improved" como "bagging" nos dan como favoritos:
    1. XGBoost
    2. K-Nearest Neighbours
    3. Random Forest

Según nuestro mejor modelo (XGBoost mejorado en hiperparámetros), que además nos permite estimar la importancia de los predictores en la variable objetivo, irían por orden:

NOx        0.656528 \
AH         0.082007 \
CO         0.055414 \
RH         0.045607 \
PT08.S2    0.035066 

Lo cual quiere decir que prácticamente ninguno de nuestros sensores es útil para predecir la concentración de NO2.

In [ ]:
###----------------< END of 'airquality_nb.ipynb'>---------------###